In [126]:
import pandas as pd 
from difflib import SequenceMatcher as SM
df_club=pd.read_csv("Clubs_data.csv")
df_meta=pd.read_csv("Metadata.csv")
df_organiser=pd.read_csv("Organisers_In_Fests.csv")
df_participants=pd.read_csv("Participants_In_Fests.csv")

In [127]:
def is_similarity(first, second, ratio):
    return SM(None, first, second).ratio() > ratio

In [128]:
for Index, row in df_club.iterrows():
    # print(Index, len(df_club.iloc[Index]["Name"]))
    df_club.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.64):
            df_club.at[Index, "Name"] = row2["Name"]
            

In [129]:
list_small_5 = []
for Index, row in df_club.iterrows():
    if len(df_club.iloc[Index]["Name"]) < 5:
        list_small_5.append(Index)


In [130]:
df_club = df_club.drop(df_club.index[list_small_5])

In [131]:
for Index, row in df_participants.iterrows():
    df_participants.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.59):
            df_participants.at[Index, "Name"] = row2["Name"]

In [132]:
list_small_7 = []
for Index, row in df_participants.iterrows():
    if len(df_participants.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)

In [133]:
df_participants = df_participants.drop(df_participants.index[list_small_7])

In [134]:
df_club

,Club_Name,Name,Event,Role
0,club_3,darien martens,club_3_event_2,Participant
1,club_2,sage schrock,club_2_event_1,Participant
2,club_3,alison mcvey,club_3_event_2,Participant
3,club_1,kiara jauregui,club_1_event_1,Participant
4,club_2,tate blum,club_2_event_3,Participant
...,...,...,...,...
1041,club_3,kalen logan,club_3_event_2,organiser_3
1042,club_3,kathrine leone,club_3_event_3,organiser_1
1043,club_1,ibrehimusku,club_1_event_2,Participant
1044,club_1,milo greenfield,club_1_event_2,Participant


In [135]:
df_organiser=df_organiser.drop(13)
df_organiser=df_organiser.drop(15)


In [136]:
print(df_club.shape)
print(df_participants.shape)
print(df_meta.shape)
print(df_organiser.shape)

(987, 4)
(4458, 3)
(500, 2)
(18, 3)


In [137]:
df_combined = pd.concat([df_club,df_organiser,df_participants])

In [138]:
df_combined.shape

(5463, 5)

In [139]:
df_combined.head()

,Club_Name,Name,Event,Role,Fest_Name
0,club_3,darien martens,club_3_event_2,Participant,NaN
1,club_2,sage schrock,club_2_event_1,Participant,NaN
2,club_3,alison mcvey,club_3_event_2,Participant,NaN
3,club_1,kiara jauregui,club_1_event_1,Participant,NaN
4,club_2,tate blum,club_2_event_3,Participant,NaN


In [140]:
df_combined.tail()

,Club_Name,Name,Event,Role,Fest_Name
4763,NaN,zoie mcdermott,fest_2_event_9,NaN,fest_2
4764,NaN,jayna rowley,fest_2_event_7,NaN,fest_2
4765,NaN,debra clevenger,fest_2_event_11,NaN,fest_2
4766,NaN,theron stroud,fest_1_event_6,NaN,fest_1
4768,NaN,kendrick valadez,fest_2_event_2,NaN,fest_2


In [141]:
df_combined.columns 
ID=[]
for i in range(1,len(df_combined)+1):
    ID.append(i)
df_combined["ID"]=ID
df_combined.tail() 

,Club_Name,Name,Event,Role,Fest_Name,ID
4763,NaN,zoie mcdermott,fest_2_event_9,NaN,fest_2,5459
4764,NaN,jayna rowley,fest_2_event_7,NaN,fest_2,5460
4765,NaN,debra clevenger,fest_2_event_11,NaN,fest_2,5461
4766,NaN,theron stroud,fest_1_event_6,NaN,fest_1,5462
4768,NaN,kendrick valadez,fest_2_event_2,NaN,fest_2,5463


In [142]:
df_combined.head()

,Club_Name,Name,Event,Role,Fest_Name,ID
0,club_3,darien martens,club_3_event_2,Participant,NaN,1
1,club_2,sage schrock,club_2_event_1,Participant,NaN,2
2,club_3,alison mcvey,club_3_event_2,Participant,NaN,3
3,club_1,kiara jauregui,club_1_event_1,Participant,NaN,4
4,club_2,tate blum,club_2_event_3,Participant,NaN,5


In [143]:
df_combined.tail()

,Club_Name,Name,Event,Role,Fest_Name,ID
4763,NaN,zoie mcdermott,fest_2_event_9,NaN,fest_2,5459
4764,NaN,jayna rowley,fest_2_event_7,NaN,fest_2,5460
4765,NaN,debra clevenger,fest_2_event_11,NaN,fest_2,5461
4766,NaN,theron stroud,fest_1_event_6,NaN,fest_1,5462
4768,NaN,kendrick valadez,fest_2_event_2,NaN,fest_2,5463


In [144]:
df_combined["Club_Name"]=df_combined["Club_Name"].fillna("Not There")
df_combined["Event"]=df_combined["Club_Name"].fillna("Not There")
df_combined["Role"]=df_combined["Role"].fillna("Not There")
df_combined["Fest_Name"]=df_combined["Fest_Name"].fillna("Not There") 

df_combined.head()



,Club_Name,Name,Event,Role,Fest_Name,ID
0,club_3,darien martens,club_3,Participant,Not There,1
1,club_2,sage schrock,club_2,Participant,Not There,2
2,club_3,alison mcvey,club_3,Participant,Not There,3
3,club_1,kiara jauregui,club_1,Participant,Not There,4
4,club_2,tate blum,club_2,Participant,Not There,5


In [145]:
df_combined.tail()

,Club_Name,Name,Event,Role,Fest_Name,ID
4763,Not There,zoie mcdermott,Not There,Not There,fest_2,5459
4764,Not There,jayna rowley,Not There,Not There,fest_2,5460
4765,Not There,debra clevenger,Not There,Not There,fest_2,5461
4766,Not There,theron stroud,Not There,Not There,fest_1,5462
4768,Not There,kendrick valadez,Not There,Not There,fest_2,5463


In [146]:
df_combined["Name"].head() 
names=df_meta["Name"].tolist()
roll=df_meta["ID"].tolist() 

df_combined.set_index("ID", drop=True, inplace=True)
dictionary_combined = df_combined.to_dict(orient="index") 





In [155]:
for i in range(dictionary_combined):
    for j in range(names):
        if dictionary_combined[i]["Name"]==names[j]:
            dictionary_combined[i]["Name"]=roll[j]

    


TypeError: 'dict' object cannot be interpreted as an integer